# Preprocessing Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.image as img
import matplotlib.pyplot as plt
from tqdm import tqdm
import os, cv2

TRAIN_DIR = ''
TEST_DIR = ''
TRAIN_LABEL_DIR = 'Data_Entry_2017.csv'
TEST_LABEL_DIR = 'Data_Entry_2017.csv'
IMG_SIZE = 256

In [ ]:
# One Hot Encode for Multiple Labels
from sklearn.preprocessing import MultiLabelBinarizer

read_data = pd.read_csv(TRAIN_LABEL_DIR)
split_labels = [items.split('|')[0] for items in read_data.Finding_Labels]
one_hot = MultiLabelBinarizer()
training_labels = one_hot.fit_transform(split_labels)
training_labels = training_labels[3001:35000]

read_data = pd.read_csv(TEST_LABEL_DIR)
split_labels = [items.split('|')[0] for items in read_data.Finding_Labels]
one_hot = MultiLabelBinarizer()
test_labels = one_hot.fit_transform(split_labels)
test_labels = test_labels[0:3000]

In [ ]:
def create_batch(directory):
    img_array = []
    for img in tqdm(os.listdir(directory)):
        path = os.path.join(directory, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        img_array.append(img)
    return img_array

In [ ]:
training_img = create_batch(TRAIN_DIR)
training_img = np.array(training_img)
training_img = np.expand_dims(training_img, axis = 3)

In [ ]:
test_img = create_batch(TEST_DIR)
test_img = np.array(test_img)
test_img = np.expand_dims(test_img, axis = 3)

# Neural Network

In [ ]:
import keras
from keras import models
from keras import layers

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(33, activation='softmax'))

#model.summary()

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(training_img, training_labels, epochs = 1, batch_size = 128)

In [ ]:
test_loss, test_acc = model.evaluate(test_img, test_labels)
test_acc